In [1]:
# Install Kaggle API
!pip install -q kaggle

In [9]:
# Import libraries
import os
from google.colab import files
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

random_state = 24

In [3]:
# Check if the API key already exists
if not os.path.exists('/root/.kaggle/kaggle.json'):
    print('Please upload your personal kaggle.json file:')

    # Prompt for upload
    uploaded = files.upload()

    # Move the file to the correct directory
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print('\nKaggle API key configured successfully.')
else:
    print('Kaggle API key is already configured.')

Please upload your personal kaggle.json file:


Saving kaggle.json to kaggle.json

Kaggle API key configured successfully.


In [4]:
# Replace with your dataset id here
KAGGLE_DATASET_ID = 'andymalinsky/rsna-2022-hdf5-subset'

print('Downloading dataset...')
!kaggle datasets download -d {KAGGLE_DATASET_ID}

Dataset URL: https://www.kaggle.com/datasets/andymalinsky/rsna-2022-hdf5-subset
License(s): CC0-1.0
 99% 3.50G/3.51G [02:09<00:00, 252MB/s]
100% 3.51G/3.51G [02:09<00:00, 29.2MB/s]


In [5]:
# Unzip and move to the local Colab storage for training
zip_name = KAGGLE_DATASET_ID.split('/')[-1] + '.zip'
!unzip -q '{zip_name}'

# This is the path the training script will use
HDF5_FILE_PATH = '/content/fracture_dataset_subset.h5'

In [6]:
# Get dataloader script from GitHub
!wget https://raw.githubusercontent.com/apmalinsky/AAI-590-Capstone/refs/heads/main/scripts/dataloader.py

--2025-11-25 06:44:49--  https://raw.githubusercontent.com/apmalinsky/AAI-590-Capstone/refs/heads/main/scripts/dataloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917 (5.8K) [text/plain]
Saving to: ‘dataloader.py’

dataloader.py       100%[===================>]   5.78K  --.-KB/s    in 0s      

2025-11-25 06:44:49 (83.9 MB/s) - ‘dataloader.py’ saved [5917/5917]



In [7]:
# Import dataloader script
from dataloader import get_dataloaders

In [8]:
# View data file
data_path = 'fracture_dataset_subset.h5'

# Open the file
with h5py.File(data_path, 'r') as f:
    # View key list, essentially our 'columns'
    print(f'Keys: {list(f.keys())}')

    # View column shapes
    print('\n--- Details ---')
    for key in f.keys():
        dataset = f[key]
        print(f'\nKey: {key}')
        print(f'Shape: {dataset.shape}')
        print(f'Dtype: {dataset.dtype}')

Keys: ['SliceNumber', 'StudyInstanceUID', 'bboxes', 'images', 'labels', 'split']

--- Details ---

Key: SliceNumber
Shape: (28868,)
Dtype: object

Key: StudyInstanceUID
Shape: (28868,)
Dtype: object

Key: bboxes
Shape: (28868, 10, 4)
Dtype: float32

Key: images
Shape: (28868, 256, 256)
Dtype: float32

Key: labels
Shape: (28868,)
Dtype: int8

Key: split
Shape: (28868,)
Dtype: int8


In [10]:
# Get data loaders for detection
batch_size = 32

train_loader, val_loader, test_loader = get_dataloaders(
    hdf5_path=data_path,
    batch_size=batch_size,
    task='classification',
    num_workers=2
)

# Check one batch
images, labels = next(iter(train_loader))
print(f"Batch shape: {images.shape}, Labels shape: {labels.shape}")

Loading dataset for task: 'classification'...
Loading pre-defined train/val/test splits...
Total samples: 28868
  Training indices:   20207
  Validation indices: 4330
  Test indices:       4331
DataLoaders created successfully.
Batch shape: torch.Size([32, 1, 256, 256]), Labels shape: torch.Size([32])


## Simple CNN Model

In [21]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # infer the linear layer input size dynamically
        self._flatten_size = None

    def _get_flatten_size(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        return x.view(x.size(0), -1).shape[1]

    def build_fc(self, sample_shape):
      device = next(self.parameters()).device
      x = torch.zeros(1, *sample_shape).to(device)
      with torch.no_grad():
          flat_size = self._get_flatten_size(x)
      self._flatten_size = flat_size
      self.fc1 = nn.Linear(flat_size, 256).to(device)
      self.fc2 = nn.Linear(256, 2).to(device)

    def forward_features(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return x

    def forward(self, x):
        if self._flatten_size is None:
            self.build_fc(x.shape[1:])
        x = self.forward_features(x)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print("Using device:", device)

Using device: cuda


## Training and Evaluation

In [22]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        preds = outputs.argmax(dim=1)
        all_preds.append(preds.detach().cpu().numpy())
        all_labels.append(labels.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    epoch_f1  = f1_score(all_labels, all_preds)

    return epoch_loss, epoch_acc, epoch_f1


def eval_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            preds = outputs.argmax(dim=1)
            all_preds.append(preds.detach().cpu().numpy())
            all_labels.append(labels.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    epoch_loss = running_loss / len(loader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    epoch_f1  = f1_score(all_labels, all_preds)

    return epoch_loss, epoch_acc, epoch_f1, all_labels, all_preds

In [24]:
num_epochs = 10
best_val_f1 = 0.0
best_state = None

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc, train_f1 = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )
    val_loss, val_acc, val_f1, y_true, y_pred = eval_one_epoch(
        model, val_loader, criterion, device
    )

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_state = model.state_dict()

    print(
        f"Epoch {epoch:02d} "
        f"Train loss {train_loss:.4f} acc {train_acc:.3f} f1 {train_f1:.3f}  "
        f"Val loss {val_loss:.4f} acc {val_acc:.3f} f1 {val_f1:.3f}"
    )

if best_state is not None:
    model.load_state_dict(best_state)
    print("Loaded best model with val F1:", best_val_f1)

Epoch 01 Train loss 0.5153 acc 0.754 f1 0.089  Val loss 0.5062 acc 0.751 f1 0.031
Epoch 02 Train loss 0.5108 acc 0.753 f1 0.109  Val loss 0.5006 acc 0.755 f1 0.075
Epoch 03 Train loss 0.5062 acc 0.755 f1 0.146  Val loss 0.4944 acc 0.758 f1 0.100
Epoch 04 Train loss 0.5005 acc 0.755 f1 0.190  Val loss 0.4956 acc 0.763 f1 0.159
Epoch 05 Train loss 0.4973 acc 0.757 f1 0.198  Val loss 0.4853 acc 0.764 f1 0.195
Epoch 06 Train loss 0.4940 acc 0.758 f1 0.219  Val loss 0.4791 acc 0.762 f1 0.154
Epoch 07 Train loss 0.4892 acc 0.759 f1 0.242  Val loss 0.4812 acc 0.769 f1 0.229
Epoch 08 Train loss 0.4879 acc 0.759 f1 0.244  Val loss 0.4757 acc 0.768 f1 0.207
Epoch 09 Train loss 0.4856 acc 0.760 f1 0.255  Val loss 0.4739 acc 0.769 f1 0.214
Epoch 10 Train loss 0.4828 acc 0.762 f1 0.263  Val loss 0.4740 acc 0.767 f1 0.214
Loaded best model with val F1: 0.22857142857142856
